In [76]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasRegressor
from keras.datasets import mnist
from sklearn.model_selection import GridSearchCV
import pprint
pp = pprint.PrettyPrinter(indent=4)

## Model Definition

In [94]:
# %load models/lstm_model.py
# import the relevant Keras modules
"""
    If network is overfitting => decrease batch size; the contrary is true for underfitting
"""
def lstm_model(neurons, optimizer, loss, activ_func, dropout): 
    
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(10,22)))
    model.add(Dropout(dropout))
    model.add(Dense(units=1))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [95]:
print(optimizers.SGD)

<class 'tensorflow.python.keras.optimizers.SGD'>


## Load Dataset

In [96]:
split_date = '2018-01-01' 

btc_final = pd.read_csv("btc_final.csv")

# Get rid of 'date' columns
training_set, test_set = btc_final[btc_final['date']<split_date], btc_final[btc_final['date']>=split_date]
training_set = training_set.drop('date', 1)
test_set = test_set.drop('date', 1)

window_len = 10
norm_cols = ['bt_close','bt_volume']

LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    LSTM_training_inputs.append(temp_set)
    
LSTM_training_outputs = (training_set['bt_close'][window_len:].values/training_set['bt_close'][:-window_len].values)-1

pred_range=10


LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    LSTM_test_inputs.append(temp_set)
LSTM_test_outputs = (test_set['bt_close'][window_len:].values/test_set['bt_close'][:-window_len].values)-1


LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

print(LSTM_training_inputs[:-pred_range].shape)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

(1446, 10, 22)


In [97]:
# training_set, test_set = btc_final[btc_final['date']<split_date], btc_final[btc_final['date']>=split_date]

LSTM_training_outputs=LSTM_training_outputs[0:LSTM_training_outputs.size-10]
print(len(LSTM_training_outputs))
# LSTM_training_outputs = LSTM_training_outputs.reshape(LSTM_training_outputs.shape[1:])

1446


## Define the Parameters

In [98]:
# [:1] is for testing
# batch_size = [1, 20, 50][:1]
# epochs = [1, 20, 50][:1]
# learning_rate = [0.1, 0.001, 0.02][:1]
batch_size = [20, 50, 100][:1]
epochs = [1, 20, 50][:1]
neurons = [10, 20][:1]
optimizer = ['sgd','adam'][:1]
loss=['mae','mean_squared_error']
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'][:1]
dropout_rate = [0.3, 0.25, 0.8][:1]

## Model Wrapper and GridSearchCV

In [99]:
# Create the wrapper and pass params to GridSearchCV
params = dict(neurons = neurons,
              optimizer = optimizer,
              loss = loss,
              activ_func = activation,
              dropout = dropout_rate)

model = KerasRegressor(build_fn=lstm_model,  verbose=2, shuffle=True, batch_size = 1,
                        epochs = 50)

models = GridSearchCV(estimator = model, param_grid=params,scoring = 'neg_mean_squared_error', n_jobs=1)

## Train the Models

In [100]:
pred_range = 10
# (1446, 10, 22)

# LSTM_training_inputs = LSTM_training_inputs[:-pred_range]
# LSTM_training_inputs.shape(1446,22)
# print(LSTM_training_inputs[:-pred_range]) # 1446
best_model = models.fit(LSTM_training_inputs[:-pred_range], LSTM_training_outputs)
print('Best model :')
pp.pprint(best_model.best_params_)

Epoch 1/50
 - 4s - loss: 0.1053
Epoch 2/50
 - 4s - loss: 0.1038
Epoch 3/50
 - 5s - loss: 0.1034
Epoch 4/50
 - 5s - loss: 0.1017
Epoch 5/50
 - 6s - loss: 0.1023
Epoch 6/50
 - 6s - loss: 0.0999
Epoch 7/50
 - 5s - loss: 0.0994
Epoch 8/50
 - 4s - loss: 0.0950
Epoch 9/50
 - 4s - loss: 0.0856
Epoch 10/50
 - 4s - loss: 0.0818
Epoch 11/50
 - 4s - loss: 0.0734
Epoch 12/50
 - 5s - loss: 0.0700
Epoch 13/50
 - 5s - loss: 0.0678
Epoch 14/50
 - 5s - loss: 0.0658
Epoch 15/50
 - 5s - loss: 0.0613
Epoch 16/50
 - 5s - loss: 0.0608
Epoch 17/50
 - 5s - loss: 0.0606
Epoch 18/50
 - 5s - loss: 0.0588
Epoch 19/50
 - 5s - loss: 0.0567
Epoch 20/50
 - 5s - loss: 0.0580
Epoch 21/50
 - 5s - loss: 0.0579
Epoch 22/50
 - 5s - loss: 0.0565
Epoch 23/50
 - 5s - loss: 0.0576
Epoch 24/50
 - 5s - loss: 0.0567
Epoch 25/50
 - 5s - loss: 0.0571
Epoch 26/50
 - 5s - loss: 0.0555
Epoch 27/50
 - 5s - loss: 0.0559
Epoch 28/50
 - 5s - loss: 0.0559
Epoch 29/50
 - 5s - loss: 0.0547
Epoch 30/50
 - 5s - loss: 0.0535
Epoch 31/50
 - 5s -

Epoch 1/50
 - 8s - loss: 0.0194
Epoch 2/50
 - 7s - loss: 0.0177
Epoch 3/50
 - 7s - loss: 0.0176
Epoch 4/50
 - 7s - loss: 0.0177
Epoch 5/50
 - 7s - loss: 0.0177
Epoch 6/50
 - 7s - loss: 0.0177
Epoch 7/50
 - 7s - loss: 0.0177
Epoch 8/50
 - 6s - loss: 0.0177
Epoch 9/50
 - 4s - loss: 0.0177
Epoch 10/50
 - 4s - loss: 0.0177
Epoch 11/50
 - 4s - loss: 0.0177
Epoch 12/50
 - 4s - loss: 0.0176
Epoch 13/50
 - 5s - loss: 0.0177
Epoch 14/50
 - 5s - loss: 0.0176
Epoch 15/50
 - 5s - loss: 0.0177
Epoch 16/50
 - 5s - loss: 0.0179
Epoch 17/50
 - 5s - loss: 0.0176
Epoch 18/50
 - 5s - loss: 0.0177
Epoch 19/50
 - 5s - loss: 0.0177
Epoch 20/50
 - 5s - loss: 0.0176
Epoch 21/50
 - 5s - loss: 0.0176
Epoch 22/50
 - 5s - loss: 0.0177
Epoch 23/50
 - 5s - loss: 0.0177
Epoch 24/50
 - 5s - loss: 0.0177
Epoch 25/50
 - 5s - loss: 0.0177
Epoch 26/50
 - 5s - loss: 0.0177
Epoch 27/50
 - 5s - loss: 0.0175
Epoch 28/50
 - 5s - loss: 0.0178
Epoch 29/50
 - 5s - loss: 0.0176
Epoch 30/50
 - 5s - loss: 0.0177
Epoch 31/50
 - 5s -